In [1]:
import os
import subprocess
import json
from datetime import datetime
from fastapi import FastAPI, HTTPException, Query
from pydantic import BaseModel
from typing import List, Dict, Any
import sqlite3
import requests
from openai import OpenAI

from fastapi import FastAPI, HTTPException, Query
from fastapi.middleware.cors import CORSMiddleware
import os
import requests
import subprocess
import json
from dotenv import load_dotenv
load_dotenv() 

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [2]:
tools = [
    {
        "name": "function",
        "type": "function",
        "function":{
            "name": "script_runner",
            "description": "Install a package and run a script from an URL with provided arguments",
            "parameters": {
                "type": "object",
                "properties": {
                    "script_url": {
                        "type": "string",
                        "description": "The URL of the script to run"
                    },
                    "args": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "List of arguments to pass to the script"
                    },   

                }, "required": ["script_url", "args"]
            }
        },
    }
]

In [3]:
AIPROXY_TOKEN = os.getenv("AIPROXY_TOKEN")


In [4]:
@app.get("/")
async def root():
    return {"message": "Hello World"}

In [5]:
@app.get("/read")
def read_file(path:str):
    try:
        with open(path, 'r') as f:
            content = f.read()
        return content
    except Exception as e:
        raise HTTPException(status_code=404, detail=str(e))
    

In [40]:
CACHE_FILE = "api_cache.json"


@app.post("/run")
def task_runner(task:str):
    url = "https://aiproxy.sanand.workers.dev/openai/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {AIPROXY_TOKEN}"
    }

    data = {   
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "user",
                "content": task
            },
            {
                "role": "system",
                "content": """
You are an assistant who has to do a variety of tasks
If your task involves running a script, you can use the script_runner tool.
If your task involves writing a code, you can use the task_runner tool.
"""
            }
        ],
        "tools": tools,
        "tool_choice": "auto"
    }

    response = requests.post(url, headers=headers, json=data)
    if not response.ok:
        raise HTTPException(status_code=response.status_code, detail=response.text)
    
    res_json = response.json()
    cache = res_json
    with open(CACHE_FILE, "w") as f:
        json.dump(cache, f)
    try:
        argu = res_json['choices'][0]['message']
    except (KeyError, IndexError) as e:
        raise HTTPException(status_code=500, detail=f"Error parsing response: {e}\nResponse: {res_json}")
    
    try:
        arguments = argu['tool_calls'][0]['function']['arguments']
        # If arguments is a string, parse it; otherwise, assume it's already a dict.
        if isinstance(arguments, str):
            func_args = json.loads(arguments)
        else:
            func_args = arguments
        script_url = func_args['script_url']
        email = func_args['args'][0]
        print(script_url, email)
        command = ["uv","run",script_url, email]
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return {"output": result.stdout, "error": result.stderr}
    except Exception as ex:
        raise HTTPException(status_code=500, detail=f"Error executing script: {ex}")

In [7]:


# # Run this only once manually in a separate cell
# import uvicorn
# uvicorn.run(app, host="0.0.0.0", port=8000, reload=False)

import uvicorn
import threading

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, reload=False)

# Start the server in a separate thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()



INFO:     Started server process [127330]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [41]:
BASE_URL = "http://127.0.0.1:8000"


# Test the run endpoint
# task_description = "Count occurrences of Monday in the dataset"
task_description = "Install uv (if required) and run https://raw.githubusercontent.com/sanand0/tools-in-data-science-public/tds-2025-01/project-1/datagen.py with 22f3002341@ds.study.iitm.ac.in as the only argument."
response = requests.post(f"{BASE_URL}/run", params={"task": task_description})
print(response.json())  # Should return cached result after first call


{'detail': 'Error executing script: string indices must be integers'}


In [42]:
response.json()

{'detail': 'Error executing script: string indices must be integers'}

In [ ]:
# To kill the threading server

import os
import signal

# Get the process ID of the current Python process
pid = os.getpid()

# Kill the process (forcefully stopping uvicorn)
os.kill(pid, signal.SIGTERM)


: 

In [43]:
CACHE_FILE = "api_cache.json"

# Load and read the JSON file
with open(CACHE_FILE, "r") as f:
    cache_data = json.load(f)

# Print all cached tasks
print(json.dumps(cache_data, indent=4))

{
    "id": "chatcmpl-B0Fx8dQzGzI9MOTipHdMmQ2lr5BxZ",
    "object": "chat.completion",
    "created": 1739401022,
    "model": "gpt-4o-mini-2024-07-18",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": null,
                "tool_calls": [
                    {
                        "id": "call_XQyICx8unZhmoOZrxQlEFu3l",
                        "type": "function",
                        "function": {
                            "name": "script_runner",
                            "arguments": "{\"script_url\":\"https://raw.githubusercontent.com/sanand0/tools-in-data-science-public/tds-2025-01/project-1/datagen.py\",\"args\":[\"22f3002341@ds.study.iitm.ac.in\"]}"
                        }
                    }
                ],
                "refusal": null
            },
            "logprobs": null,
            "finish_reason": "tool_calls"
        }
    ],
    "usage": {
        "prompt_t

In [44]:
arguments = cache_data['choices'][0]['message']['tool_calls'][0]['function']['arguments']
# If arguments is a string, parse it; otherwise, assume it's already a dict.
if isinstance(arguments, str):
    func_args = json.loads(arguments)
else:
    func_args = arguments
script_url = func_args['script_url']
email = func_args['args'][0]

In [45]:
script_url

'https://raw.githubusercontent.com/sanand0/tools-in-data-science-public/tds-2025-01/project-1/datagen.py'

In [39]:
email

'22f3002341@ds.study.iitm.ac.in'